In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
!pip install folium
!pip install geopandas

In [ ]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

In [ ]:
# Get data of labor
sex_work = pd.read_stata( r'C:\Users\HP\Documents\GitHub\QLAB_Summer_Python\_data\8_trab_sex_20.dta' )
sex_work[ 'dpt_code' ] = sex_work.cod_ubigeo.str[ :2 ].copy()
sex_work[ 'prov_code' ] = sex_work.cod_ubigeo.str[ :4 ].copy()
sex_work.head()

In [ ]:
# Sex work
# Me quedo con la data de mujeres
women_work = sex_work[ sex_work.sex == 'Mujer' ].copy().reset_index( drop = True )
women_work.head()

In [ ]:
# get data from lima
## Convirtiendo la columna llamada month en un objeto de fecha y hora
women_work[ 'month' ] = pd.to_datetime( women_work.month , format = '%B' ) \
                                    .dt.strftime( '%m' ) \
                                    .astype( int )

In [ ]:
## Sort by department and month (quedandonos con la info departamental y los meses, luego actualizando en la misma base original)
women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

# Get the total number of women workers by dpt (sumando empl, para esto se agrupan teniendo en cuenta dpt y month)
dpt_women_work = women_work.groupby( [ 'dpt_code', 'month'], as_index = False )[['empl']] \
                            .sum() \
                            .rename( columns = {'empl' :'women_empl'})

# Sort by dpt code and month (ordenar)
dpt_women_work.sort_values([ 'dpt_code', 'month'], inplace = True )
dpt_women_work.head()

In [ ]:
# consiguiendo el promedio de mujeres empleadas por departamento
df2 = dpt_women_work.groupby( ['dpt_code'],as_index = False )[['women_empl']].mean()
df2.head(25)

In [ ]:
# Starting with the Shapefile

base = open(r'C:\Users\HP\Documents\GitHub\QLAB_Summer_Python\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']

dpt_shp = gpd.read_file( r'C:\Users\HP\Documents\GitHub\QLAB_Summer_Python\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp', encoding = charenc)
dpt_shp


In [ ]:
# Mergeamos nuestra base de datos del promedio de mujeres empleadas por departamentos con el shapefile, la columna en común es dpt_code)
df3 = dpt_shp.merge( df2, left_on = 'CCDD', right_on = 'dpt_code'  )
df3

In [ ]:
# First Graph
df3['women_empl'] = df3['women_empl'].astype(str).astype(float)
df2['women_empl'] = df2['women_empl'].astype(str).astype(float)


# Se usarán las coordenadas de palacio de gobierno como centroide

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)


fm.Choropleth(
    geo_data=df3,
    data=df2,
    columns=['dpt_code', 'women_empl'],
    key_on="feature.properties.dpt_code",
    fill_color="Reds",
    fill_opacity=0.8,
    ).add_to(z)

z

In [ ]:
# Second Graph
df3['women_empl'] = df3['women_empl'].astype(str).astype(float)
df2['women_empl'] = df2['women_empl'].astype(str).astype(float)


# Se usarán las coordenadas de palacio de gobierno como centroide

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z2 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)


choropleth=fm.Choropleth(
    geo_data=df3,
    data=df2,
    columns=['dpt_code', 'women_empl'],
    key_on="feature.properties.dpt_code",
    fill_color="Reds",
    fill_opacity=0.8,
    legend_name= "Number of Employers",
    bins = [675.333333, 20000, 40000, 60000, 100000, 656477.250000]
    ).add_to(z2)

choropleth.geojson.add_child(fm.features.GeoJsonTooltip(['dpt_code', 'women_empl']))

z2

In [ ]:
# Supongamos que deseas eliminar la fila con índice 14 (0-indexed)
index_to_remove = 14

# Elimina la fila especificada por el índice
df4 = df3.drop(index=index_to_remove)

# Asegúrate de que df4 sea tu nuevo DataFrame
df4.reset_index(drop=True, inplace=True)

# Muestra el nuevo DataFrame
df4

In [ ]:
# Asumiendo que deseas eliminar la fila con índice 14 (0-indexed)
index_to_remove = 14

# Elimina la fila especificada por el índice
df5 = df2.drop(index_to_remove)

# Asegúrate de que df2 sea tu nuevo DataFrame
df5.reset_index(drop=True, inplace=True)

# Muestra el nuevo DataFrame
df5.head(25)

In [ ]:
df4['women_empl'] = df4['women_empl'].astype(str).astype(float)
df5['women_empl'] = df5['women_empl'].astype(str).astype(float)


# Se usarán las coordenadas de palacio de gobierno como centroide

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z3 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)


fm.Choropleth(
    geo_data=df4,
    data=df5,
    columns=['dpt_code', 'women_empl'],
    key_on="feature.properties.dpt_code",
    fill_color="Reds",
    fill_opacity=0.8,
    ).add_to(z3)

z3

In [ ]:
df6 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()
df6

In [ ]:
df6[ 'prop_wom' ] = ( df6.Mujer * 100 / df6.Hombre )
df7 = dpt_shp.merge( df6, left_on = 'CCDD', right_on = 'dpt_code'  )
df7

In [ ]:
import folium as fm

# Asumiendo que df5 es un DataFrame con datos geoespaciales
# Asegúrate de que df5 tiene una columna 'month' con los nombres de los meses

# Lista de meses únicos en el DataFrame
unique_months = df7['month'].unique()

# Se usarán las coordenadas de palacio de gobierno como centroide
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

# Crear un objeto de mapa inicial
z4 = fm.Map(location=[lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start=zoom_start)

# Bucle para crear mapas individuales para cada mes
for month in unique_months:
    # Filtrar el DataFrame por mes
    df_month = df7[df7['month'] == month]
    
    # Crear un objeto de mapa para el mes actual
    m = fm.Map(location=[lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start=zoom_start)
    
    # Agregar un Choropleth al mapa para el mes actual
    fm.Choropleth(
        geo_data=df7,
        data=df_month,
        columns=['dpt_code', 'prop_wom'],
        key_on="feature.properties.dpt_code",
        fill_color="Reds",
        fill_opacity=0.8,
    ).add_to(m)

    # Mostrar el mapa del mes actual
    display(m)

In [ ]:
# Eliminar todas las filas que tienen 'LIMA' en la columna 'Region' (o la columna relevante)
df8 = df7[df7['NOMBDEP'] != 'LIMA']

# Asegurar que los índices sean consecutivos después de la eliminación
df8.reset_index(drop=True, inplace=True)

# Mostrar el nuevo DataFrame df8
df8

In [ ]:
# Copiar los DataFrames para evitar la SettingWithCopyWarning
df6_copy = df6.copy()
df8_copy = df8.copy()

# Convertir la columna 'prop_wom' a tipo float
df6_copy['prop_wom'] = df6_copy['prop_wom'].astype(float)
df8_copy['prop_wom'] = df8_copy['prop_wom'].astype(float)

# Se usarán las coordenadas de palacio de gobierno como centroide
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

# Crear un objeto de mapa
z5 = fm.Map(location=[lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start=zoom_start)

# Crear el Choropleth y agregarlo al mapa
choropleth = fm.Choropleth(
    geo_data=df8_copy,
    data=df6_copy,
    columns=['dpt_code', 'prop_wom'],
    key_on="feature.properties.dpt_code",
    fill_color="Reds",
    fill_opacity=0.8,
    legend_name="Women Proportion",
    bins=[17.01, 20, 30, 40, 50, 100]
).add_to(z5)

# Mostrar el mapa
z5